In [ ]:
import cv2
import torch
import numpy as np
from sklearn.metrics.pairwise import cosine_similarity

# Load the YOLO model (replace 'best.pt' with your trained weights)
model = torch.hub.load('ultralytics/yolov5', 'custom', path='best.pt')
model.conf = 0.4  # confidence threshold

# Feature extractor using color histogram
def extract_feature(crop):
    crop = cv2.resize(crop, (64, 128))
    hsv = cv2.cvtColor(crop, cv2.COLOR_BGR2HSV)
    hist = cv2.calcHist([hsv], [0, 1, 2], None, [8, 8, 8],
                        [0, 180, 0, 256, 0, 256])
    return cv2.normalize(hist, hist).flatten()

# Track storage
tracks = {}
next_id = 0
similarity_threshold = 0.7  # Tune this based on your model

# Detect players in frame
def detect_players(frame):
    results = model(frame)[0]
    players = []
    for box in results.boxes:
        cls_id = int(box.cls)
        if cls_id == 0 or cls_id == 2:  # player classes (adjust based on your model)
            x1, y1, x2, y2 = map(int, box.xyxy[0])
            crop = frame[y1:y2, x1:x2]
            if crop.size == 0: continue
            feature = extract_feature(crop)
            players.append({'bbox': (x1, y1, x2, y2), 'feature': feature})
    return players

# Match players to existing IDs
def match_players(detections, tracks):
    matched_ids = []
    global next_id

    for det in detections:
        best_match = None
        best_score = 0

        for pid, data in tracks.items():
            score = cosine_similarity([det['feature']], [data['feature']])[0][0]
            if score > best_score and score > similarity_threshold:
                best_match = pid
                best_score = score

        if best_match is not None:
            matched_ids.append(best_match)
            tracks[best_match] = det  # update track
        else:
            matched_ids.append(next_id)
            tracks[next_id] = det
            next_id += 1

    return matched_ids

# Process video
def reidentify_players(video_path):
    cap = cv2.VideoCapture(video_path)
    width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
    height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
    fps = int(cap.get(cv2.CAP_PROP_FPS))

    out = cv2.VideoWriter("output_reid.mp4", cv2.VideoWriter_fourcc(*'mp4v'), fps, (width, height))

    while True:
        ret, frame = cap.read()
        if not ret:
            break

        detections = detect_players(frame)
        ids = match_players(detections, tracks)

        for det, pid in zip(detections, ids):
            x1, y1, x2, y2 = det['bbox']
            cv2.rectangle(frame, (x1, y1), (x2, y2), (0, 255, 0), 2)
            cv2.putText(frame, f"ID: {pid}", (x1, y1 - 10),
                        cv2.FONT_HERSHEY_SIMPLEX, 0.6, (0, 255, 0), 2)

        out.write(frame)
        cv2.imshow("Player ReID", frame)
        if cv2.waitKey(1) == ord('q'):
            break

    cap.release()
    out.release()
    cv2.destroyAllWindows()

# Example run
reidentify_players("15sec_input_720p.mp4")


SyntaxError: invalid syntax (261450544.py, line 63)